In [1]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import requests
import os
import zipfile
import rasterio
from rasterio.merge import merge
from tqdm import tqdm

# !pip install pyrosm
# !pip install osmium
import osmnx as ox
import pyrosm

import osmium
import shapely.wkb as wkblib

C:\Users\Ondrej\miniconda3\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\Ondrej\miniconda3\lib\site-packages\osmnx\projection.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/

# Street data

### Netherlands

In [2]:
region = "Amsterdam"
G = ox.graph_from_place(region, network_type="all") # get all networks for the given region
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G) # convert the graph to a GeoDataFrame

# # reproject to the matching projection with dtm and dsm
# gdf_nodes = gdf_nodes.to_crs('EPSG:28992')
# gdf_edges = gdf_edges.to_crs('EPSG:28992')

In [10]:
import timeit
import osmnx as ox

# Define the code as a function
def run_code():
    region = "Amsterdam"
    G = ox.graph_from_place(region, network_type="all")
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)

execution_time = timeit.timeit(run_code, number=1)

# Print the elapsed time
print(f"Elapsed time: {execution_time} seconds")

Elapsed time: 133.47383660000014 seconds


In [ ]:
# filter down to only geometries
edges_geometry = gdf_edges[["geometry"]]

# save the file as gpkg
edges_output_file = 'ned_edges.gpkg'
edges_geometry.to_file(edges_output_file, driver='GPKG')

# PBF file to Gdf

In [2]:
pbf_file = "C:/Users/Ondrej/2.5D-GreenViewIndex-Netherlands/Data collection/street_data/netherlands-latest.osm.pbf"

osm = pyrosm.OSM(pbf_file)
print('osm type:', type(osm))

osm type: <class 'pyrosm.pyrosm.OSM'>


In [3]:
class StreetsHandler(osmium.SimpleHandler):
    def __init__(self):
        osmium.SimpleHandler.__init__(self)
        self.num_nodes = 0
        self.num_relations = 0
        self.num_ways = 0
        self.street_relations = []
        self.street_relation_members = []
        self.street_ways = []
        # A global factory that creates WKB from a osmium geometry
        self.wkbfab = osmium.geom.WKBFactory()

    def way(self, w):
        if w.tags.get("highway") is not None and w.tags.get("name") is not None:
            try:
                wkb = self.wkbfab.create_linestring(w)
                geo = wkblib.loads(wkb, hex=True)
            except:
                return
            row = { "w_id": w.id, "geo": geo }

            for key, value in w.tags:
                row[key] = value

            self.street_ways.append(row)
            self.num_ways += 1

    def relation(self, r):
        if r.tags.get("type") == "associatedStreet" and r.tags.get("name") is not None:
            row = { "r_id": r.id }
            for key, value in r.tags:
                row[key] = value
            self.street_relations.append(row)

            for member in r.members:
                self.street_relation_members.append({ 
                    "r_id": r.id, 
                    "ref": member.ref, 
                    "role": member.role, 
                    "type": member.type, })
                self.num_relations += 1

In [ ]:
osm_file = "C:/Users/Ondrej/2.5D-GreenViewIndex-Netherlands/Data collection/street_data/netherlands-latest.osm.pbf"
handler = StreetsHandler()

# start data file processing
handler.apply_file(osm_file, locations=True, idx='flex_mem')
street_ways_df = pd.DataFrame(handler.street_ways)

In [5]:
street_ways_df_filtered = street_ways_df[['w_id','geo','highway','name']]
street_ways_df_filtered

,w_id,geo,highway,name
0,2161182,"LINESTRING (5.1151418 52.0683809, 5.1159473 52...",footway,Jutfaseweg
1,2952999,"LINESTRING (5.1177934 52.0776909, 5.1181293 52...",footway,Pieter Bernagiestraat
2,2953564,"LINESTRING (5.1039733 52.0904678, 5.1039554 52...",cycleway,Timorkade
3,2956818,"LINESTRING (5.1249625 52.0796443, 5.1250297 52...",pedestrian,Alkhof
4,2976724,"LINESTRING (5.1235265 52.0818684, 5.1235128 52...",pedestrian,Twijnstraat aan de Werf
...,...,...,...,...
1118027,1175423565,"LINESTRING (5.674871 53.0252689, 5.6756754 53....",cycleway,Oppenhuizerweg
1118028,1175423566,"LINESTRING (5.6752232 53.0251155, 5.6751623 53...",secondary,Oppenhuizerweg
1118029,1175423567,"LINESTRING (5.6752232 53.0251155, 5.6760735 53...",secondary,Oppenhuizerweg
1118030,1175423568,"LINESTRING (5.6764168 53.0241103, 5.6765425 53...",secondary,Oppenhuizerweg


In [7]:
sum(street_ways_df_filtered['highway'] == 'pedestrian')

19369

In [9]:
street_ways_df_filtered['highway'].value_counts()

residential               363950
unclassified              216009
cycleway                  115742
tertiary                  106946
service                    62637
secondary                  56766
footway                    49924
primary                    36041
track                      26867
living_street              25811
pedestrian                 19369
path                       12565
trunk                       5693
construction                4584
busway                      4498
platform                    2080
motorway                    2043
steps                       1611
proposed                    1592
primary_link                 852
secondary_link               676
motorway_link                474
trunk_link                   414
tertiary_link                269
services                     227
raceway                      106
corridor                      93
bridleway                     82
rest_area                     72
road                          16
bus_stop  

In [19]:
import geopandas as gpd

# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(street_ways_df_filtered, geometry='geo')

# Save as GeoPackage
gdf.to_file('netherlands_streets.gpkg', driver='GPKG')

In [2]:
gdf = gpd.read_file('street_data/netherlands_streets.gpkg')

# Print the GeoDataFrame
print(gdf)

               w_id     highway                     name  \
0           2161182     footway               Jutfaseweg   
1           2952999     footway    Pieter Bernagiestraat   
2           2953564    cycleway                Timorkade   
3           2956818  pedestrian                   Alkhof   
4           2976724  pedestrian  Twijnstraat aan de Werf   
...             ...         ...                      ...   
1118027  1175423565    cycleway           Oppenhuizerweg   
1118028  1175423566   secondary           Oppenhuizerweg   
1118029  1175423567   secondary           Oppenhuizerweg   
1118030  1175423568   secondary           Oppenhuizerweg   
1118031  1175423569     primary        Stadsrondweg-Oost   

                                                  geometry  
0          LINESTRING (5.11514 52.06838, 5.11595 52.06815)  
1        LINESTRING (5.11779 52.07769, 5.11813 52.07759...  
2        LINESTRING (5.10397 52.09047, 5.10396 52.09046...  
3        LINESTRING (5.12496 52.079

### City level

In [2]:
cities = ["Amsterdam, Netherlands", "Rotterdam, Netherlands", "Utrecht, Netherlands", "The Hague, Netherlands"]

all_gdf_nodes = []
all_gdf_edges = []

# Iterate over cities with progress tracking
total_cities = len(cities)
for i, city in enumerate(cities):
    G = ox.graph_from_place(city, network_type="all")
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)

    gdf_nodes = gdf_nodes.to_crs('EPSG:28992') # reproject to the matching projection with dtm and dsm
    gdf_edges = gdf_edges.to_crs('EPSG:28992')

    all_gdf_nodes.append(gdf_nodes)
    all_gdf_edges.append(gdf_edges)
    # progress
    progress = f"{i+1}/{total_cities}"
    print(f"Processing city {city} ({progress})")

# merge into a geodataframe
merged_gdf_nodes = gpd.GeoDataFrame(pd.concat(all_gdf_nodes, ignore_index=True))
merged_gdf_edges = gpd.GeoDataFrame(pd.concat(all_gdf_edges, ignore_index=True))

In [22]:
# filter down to only geometries
edges_geometry = merged_gdf_edges[["geometry"]]

# save the file as gpkg
edges_output_file = 'merged_edges.gpkg'
edges_geometry.to_file(edges_output_file, driver='GPKG')

# DSM/DTM files from AHN3

The below download DSM/DTM tiles and merges them into a single .tif file. The input url links for the input text file can be found from the following website:https://app.pdok.nl/rws/ahn3/download-page/#, where tile url links look as follows: https://download.pdok.nl/rws/ahn3/v1_0/05m_dsm/R_02EZ1.ZIP

In [5]:
import os
import requests
import os
import rasterio

def download_tiles(input_directory, working_dir):
    # Read tile URLs from the input directory
    with open(input_directory, 'r') as file:
        tile_urls = file.read().splitlines()

    os.makedirs(working_dir, exist_ok=True)
    # Progress
    total_tiles = len(tile_urls) 
    downloaded_tiles = 0

    for url in tile_urls:
        file_name = url.split('/')[-1]
        output_path = os.path.join(working_dir, file_name)
        response = requests.get(url)
        with open(output_path, 'wb') as f:
            f.write(response.content)
        print("Downloaded", file_name)
        print("Saved to", os.path.abspath(output_path))

        # Update the progress
        downloaded_tiles += 1
        print("Progress: {}/{}".format(downloaded_tiles, total_tiles))

        
def extract_files(working_directory, output_directory):
    os.makedirs(output_directory, exist_ok=True)  # Create the 'extracted' directory

    for file in os.listdir(working_directory):
        if zipfile.is_zipfile(os.path.join(working_directory, file)):
            with zipfile.ZipFile(os.path.join(working_directory, file)) as item:
                item.extractall(output_directory)
                print("Extracted files from:", file)

    print("Files extracted and saved to:", output_directory)
    


def merge_tif_files(extracted_directory, output_directory, filename):
    os.makedirs(output_directory, exist_ok=True)

    # get tiles into a list
    tif_files = []
    for root, dirs, files in os.walk(extracted_directory):
        tif_files += [os.path.join(root, file) for file in files if file.endswith('.TIF')]

    # merge files
    merged_arr, out_trans = merge(tif_files)

    # Output file path
    output_file = os.path.join(output_directory, f'{filename} merged.tif')

    # Output and save the file
    with rasterio.open(tif_files[0]) as src:
        meta = src.meta

    meta.update({"driver": "GTiff",
                 "height": merged_arr.shape[1],
                 "width": merged_arr.shape[2],
                 "transform": out_trans})

    with rasterio.open(output_file, "w", **meta) as dst:
        dst.write(merged_arr)
    
input_dir = 'C:/Users/Ondrej/2.5D-GreenViewIndex-Netherlands/Data collection/tiles_list_dsm.txt'
working_dir = 'C:/Users/Ondrej/2.5D-GreenViewIndex-Netherlands/Data collection/tiles_dtm'
output_dir = 'C:/Users/Ondrej/2.5D-GreenViewIndex-Netherlands/Data collection/tiles_dtm/extracted'

# download_tiles(input_dir, working_dir)
# extract_files(working_dir, output_dir)
merge_tif_files(output_dir, working_dir, 'DSM')

In [10]:
working_directory = 'C:/Users/Ondrej/2.5D-GreenViewIndex-Netherlands/Data collection/tiles_dtm'
output_directory = 'C:/Users/Ondrej/2.5D-GreenViewIndex-Netherlands/Data collection/extracted'

os.makedirs(output_directory, exist_ok=True)  # Create the 'extracted' directory

for file in os.listdir(working_directory):
    if zipfile.is_zipfile(os.path.join(working_directory, file)):
        with zipfile.ZipFile(os.path.join(working_directory, file)) as item:
            item.extractall(output_directory)
            print("Extracted files from:", file)

print("Files extracted and saved to:", output_directory)

Extracted files from: M5_01CZ1.ZIP
Extracted files from: M5_01CZ2.ZIP
Extracted files from: M5_01DZ1.ZIP
Extracted files from: M5_01DZ2.ZIP
Extracted files from: M5_01GN1.ZIP
Extracted files from: M5_01GN2.ZIP
Extracted files from: M5_01GZ1.ZIP
Extracted files from: M5_01GZ2.ZIP
Extracted files from: M5_01HN1.ZIP
Extracted files from: M5_01HN2.ZIP
Extracted files from: M5_01HZ1.ZIP
Extracted files from: M5_01HZ2.ZIP
Extracted files from: M5_02CN1.ZIP
Extracted files from: M5_02CN2.ZIP
Extracted files from: M5_02CZ1.ZIP
Extracted files from: M5_02CZ2.ZIP
Extracted files from: M5_02DN1.ZIP
Extracted files from: M5_02DZ1.ZIP
Extracted files from: M5_02DZ2.ZIP
Extracted files from: M5_02EZ1.ZIP
Extracted files from: M5_02EZ2.ZIP
Extracted files from: M5_02FZ1.ZIP
Extracted files from: M5_02FZ2.ZIP
Extracted files from: M5_02GN1.ZIP
Extracted files from: M5_02GN2.ZIP
Extracted files from: M5_02GZ1.ZIP
Extracted files from: M5_02GZ2.ZIP
Extracted files from: M5_02HN1.ZIP
Extracted files from

In [11]:
extracted_directory = 'C:/Users/Ondrej/2.5D-GreenViewIndex-Netherlands/Data collection/extracted'
output_directory = 'C:/Users/Ondrej/2.5D-GreenViewIndex-Netherlands/Data collection'
output_file = os.path.join(output_directory, 'merged.tif')

os.makedirs(output_directory, exist_ok=True) 

# get all the tiles into a list
tif_files = []
for root, dirs, files in os.walk(extracted_directory):
    tif_files += [os.path.join(root, file) for file in files if file.endswith('.TIF')]

# total
total_files = len(tif_files)
print(total_files)

# merge the files
merged_arr, out_trans = merge(tif_files)

# output and save the file 
with rasterio.open(tif_files[0]) as src:
    meta = src.meta

meta.update({"driver": "GTiff",
             "height": merged_arr.shape[1],
             "width": merged_arr.shape[2],
             "transform": out_trans})

with rasterio.open(output_file, "w", **meta) as dst:
    dst.write(merged_arr)

1371
